data = datasets.load_iris()
x_data = data.data
y_data = data.target

x_data.shape

y_data.shape

data = np.hstack((x_data, np.reshape(y_data,(150,-1))))

data.shape

np.random.shuffle(data)

x_data = pd.DataFrame(data[:,:-1], columns=['feature1', 'feature2','feature3','feature4'])

y_data = pd.DataFrame(data[:,-1], columns=['label'])

x_data.head()

y_data.head()

x_train = x_data.loc[:124]

x_train.shape

y_train = y_data.loc[:124]

y_train.shape

x_test = x_data.loc[125:]
y_test = y_data.loc[125:]

x_test.shape, y_test.shape

def label(val, *boundaries):
    if (val < boundaries[0]):
        return 'a'
    elif (val < boundaries[1]):
        return 'b'
    elif (val < boundaries[2]):
        return 'c'
    else:
        return 'd'

def toLabel(dff, old_feature_name):
    second = dff[old_feature_name].mean()
    minimum = dff[old_feature_name].min()
    first = (minimum + second)/2
    maximum = dff[old_feature_name].max()
    third = (maximum + second)/2
    return dff[old_feature_name].apply(label, args= (first, second, third))

for cols in x_data.columns:
    x_data[cols] = toLabel(x_data, cols)

def predict(tree, x_test):
    return traverse(tree, x_test)

x_test.head()

y_test.head()

def get_accuracy(tree,x_test, y_test):
    sum = 0
    for ix in range(x_test.shape[0]):
        sum += (predict(tree,x_test.iloc[ix]) == y_test.iloc[ix])
    return sum/x_test.shape[0]

x_data.head()

In [ ]:
from sklearn import datasets
import pandas as pd
import numpy as np
import math
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv('test_dr.csv')#importing the dataframe 

In [ ]:
train = data.iloc[:,:-1]#slicing tthe dataframe for the target and the tarining data 

In [ ]:
target = data.iloc[:,-1]

In [ ]:
train

In [ ]:
#making the class node for keeping the different values for the each node
class node:
    def __init__(self,n,ent,pent,l,df,y):
        self.name=n
        self.bf=None
        self.entropy = ent
        self.parentEntropy = pent
        self.level=l
        self.dataframe = df
        self.target = y
        self.children = []
    def add_child(self, obj):
        self.children.append(obj)
        
    def getentropy(self):
        return self.entropy
    def getlevel(self):
        return self.level
    

In [ ]:
#function used to calculate the value of a given type in a partiular feature 
def countTrue(feature_name,val):
    x = df[feature_name] == val
    count = 0
    for i in x:
        if i == True:
            count+=1
    return count

In [ ]:
# just testing the pieces of the code here 
df=train
y=target
s=list(y)
pc = pd.value_counts(y)
pc = np.array(pc)
pc

In [ ]:
# it would calculate the entropy based on the ouput coloumn y--by the count of the distinct values in the y for the value 
def entropyCalc(y):
    pc =pd.value_counts(y)
    pc = np.array(pc)
    total = pc.sum()

    entropy = 0
    for i in pc:
       
        if(i>0):
            entropy = entropy + (-1*(i/total)*math.log((i/total),2))
        
    return entropy   

In [ ]:
def weightedEntropy(df,feature,y):
    

    values = set(df[feature])
    # calculating the fraction of each type in a given feature 
    weight = {}
    for i in values:
        weight[i] = countTrue(feature,i)/df[feature].shape[0]
   
    # putting entropy corresponding to distinct values in a certain feature in dictionary called entropy
    
    entropy = {}    
    for i in values:
        entropy[i] = entropyCalc(y[df[feature]==i])
  
    
    #calculating weighted entropy... 
        
    weighted_entropy = 0
    
    for i in values:
        weighted_entropy = weighted_entropy + (weight[i]*entropy[i])
    
    return weighted_entropy
    
    

In [ ]:
def informationGain(parent_entropy, child_entropy):
    return parent_entropy - child_entropy
# information gains includes the difference in the parent entropy to the child node entropy

In [ ]:
# calculate the count of distinct values in v
def y_val(y):
    s=list(y)
    ans=set()
    for i in s:
        ans.add(i)
    return len(ans)

In [ ]:
def countY(lst, x):
    count = 0
    for ele in lst:
        if (ele == x):
            count = count + 1
    return count

In [ ]:
def build_tree(df,y, unused_features,cur_node):
    #base case
    # 1. unused is empty
    # 2. y contains only one distinct value
    if(len(unused_features)==0 or y_val(y)==1):
         return

        
    best_feature = ""
    #print(unused_features)
    
    max_info_gain = 0
    
    for f in unused_features:
        
        weighted_entropy = weightedEntropy(df,f,y)
        
        info_gain = informationGain(cur_node.parentEntropy, weighted_entropy)
        if(info_gain > max_info_gain):
            max_info_gain = info_gain
            best_feature = f
            
   # here you should know the best feature
   # print it out
   # print("Best Feature ", best_feature)  
    possible_values = set(df[best_feature])
    cur_node.bf= best_feature
    for val in possible_values:
        
        new_y=y[df[best_feature]==val]
        new_df=df[df[best_feature]==val]
        new_ent=entropyCalc(new_y)
            
        new_node=node(val,entropyCalc(new_y),cur_node.getentropy(),cur_node.getlevel()+1,new_df,new_y)
            
        cur_node.add_child(new_node)
        
    # remove best feature from unused features    
    unused_features.remove(best_feature)
   # print(unused_features)
        
    # loop over possible values of best feature    
    for child in cur_node.children:
        new_df = child.dataframe
        new_y=child.target
        
        # call build tree recursively
        build_tree(new_df,new_y,unused_features,child)
       



In [ ]:
def printtree(cur_node):
    for c in cur_node.children:
        print("Level ",c.level)
        
        ss=list(c.target)
        f=False
        if(c.bf !=None):
                f=True
            
        
        possible_outputs = set(c.target[ss])
        
        lst=list(c.target[ss])
        for val in possible_outputs:
            print("Count of ",val," = ",countY(lst,val))
        print("Current Entropy  is = ",c.getentropy())
        
        if(f==True):
            weighted_entropy = weightedEntropy(c.dataframe,c.bf,c.target)
            info_gain = informationGain(c.entropy, weighted_entropy)
            print("Splitting on feature ",cur_node.bf," with gain ratio ",info_gain)
            print()
        elif(f==False):
            print("Reached leaf Node")
            print()
        printtree(c)
    

In [ ]:
df=train
y=target

unused_features = set(df.columns)
#making of the root node here 
cur_node=node('root',entropyCalc(y),entropyCalc(y),0,df,y)
build_tree(df, y ,unused_features,cur_node)

s=list(cur_node.target)
# printing the root node here
print("Level ",cur_node.getlevel())
possible_outputs = set(s)
print(possible_outputs)
lst=list(s)
for val in possible_outputs:
    print("Count of ",val," = ",countY(lst,val))
print("Current Entropy  is = ",cur_node.getentropy())
weighted_entropy = weightedEntropy(cur_node.dataframe,cur_node.bf,cur_node.target)
info_gain = informationGain(cur_node.entropy, weighted_entropy)
print("Splitting on feature ",cur_node.bf," with gain ratio ",info_gain)
print()
# calling the function to call the print function for the rest of the nodes out there 
printtree(cur_node)

